# Finding Edge Cases

This notebook illustrates the detection and mitigation of certain edge cases in the `specprod` database.

In [2]:
#
# Imports
#
import os
import itertools
from sqlalchemy import __version__ as sqlalchemy_version
from sqlalchemy import and_
from sqlalchemy.sql import func
#
# DESI software
#
from desiutil.log import get_logger, DEBUG
from specprodDB import __version__ as specprodDB_version
import specprodDB.load as db
#
# Set the spectroscopic production run.
#
specprod = os.environ['SPECPROD'] = 'fuji'  # Change this to 'guadalupe' if needed.
#
# Working directory.
#
workingdir = os.getcwd()
print(f'sqlalchemy=={sqlalchemy_version}')
print(f'specprodDB=={specprodDB_version}')

sqlalchemy==1.4.46
specprodDB==0.9.0


In [3]:
#
# For much more output, use DEBUG/verbose mode.
#
# db.log = get_logger(DEBUG)
# postgresql = db.setup_db(schema=specprod, hostname='specprod-db.desi.lbl.gov', username='desi', verbose=True)
db.log = get_logger()
postgresql = db.setup_db(schema=specprod, hostname='specprod-db.desi.lbl.gov', username='desi')

## Finding Anomalous Targeting

We are trying to identify objects that appear on multiple tiles that have the same `targetid` and `survey`, but different targeting bits on different tiles. However, in principle, we don't care about cases where the `targetid`, `tileid` only appears as a potential target.

Let's get the set of `targetid` for a particular `survey` and `program` that appear on more than one tile:

```sql
SELECT t.targetid
    FROM fuji.target AS t JOIN fuji.fiberassign AS f ON t.targetid = f.targetid AND t.tileid = f.tileid
    WHERE t.survey = 'sv1' AND t.program = 'dark'
    GROUP BY t.targetid HAVING COUNT(t.tileid) > 1;
```

In [4]:
observed_multiple_tiles = db.dbSession.query(db.Target.targetid).join(db.Fiberassign, and_(db.Target.targetid == db.Fiberassign.targetid, db.Target.tileid == db.Fiberassign.tileid)).filter(db.Target.survey == 'sv1').filter(db.Target.program == 'dark').group_by(db.Target.targetid).having(func.count(db.Target.tileid) > 1)
print(observed_multiple_tiles)

SELECT fuji.target.targetid AS fuji_target_targetid 
FROM fuji.target JOIN fuji.fiberassign ON fuji.target.targetid = fuji.fiberassign.targetid AND fuji.target.tileid = fuji.fiberassign.tileid 
WHERE fuji.target.survey = %(survey_1)s AND fuji.target.program = %(program_1)s GROUP BY fuji.target.targetid 
HAVING count(fuji.target.tileid) > %(count_1)s


We will call the result of this query `observed_multiple_tiles`. Now let's find the distinct pairs of `targetid`, `sv1_desi_target` from this set:

```sql
SELECT DISTINCT targetid, sv1_desi_target
    FROM fuji.target WHERE targetid IN (observed_multiple_tiles) AND survey = 'sv1' AND program = 'dark';
```

In [5]:
distinct_target = db.dbSession.query(db.Target.targetid, db.Target.sv1_desi_target).filter(db.Target.targetid.in_(observed_multiple_tiles)).filter(db.Target.survey == 'sv1').filter(db.Target.program == 'dark').distinct().subquery()
print(distinct_target)

SELECT DISTINCT fuji.target.targetid, fuji.target.sv1_desi_target 
FROM fuji.target 
WHERE fuji.target.targetid IN (SELECT fuji.target.targetid 
FROM fuji.target JOIN fuji.fiberassign ON fuji.target.targetid = fuji.fiberassign.targetid AND fuji.target.tileid = fuji.fiberassign.tileid 
WHERE fuji.target.survey = :survey_1 AND fuji.target.program = :program_1 GROUP BY fuji.target.targetid 
HAVING count(fuji.target.tileid) > :count_1) AND fuji.target.survey = :survey_2 AND fuji.target.program = :program_2


We will call the result of this query `distinct_target`.  Next we eliminate cases where targetid only appears once in distinct_target:

```sql
SELECT targetid
    FROM (distinct_target) AS dt GROUP BY targetid HAVING COUNT(sv1_desi_target) > 1;
```

In [6]:
multiple_target = db.dbSession.query(distinct_target.c.targetid).group_by(distinct_target.c.targetid).having(func.count(distinct_target.c.sv1_desi_target) > 1)
print(multiple_target)
multiple_target.count()

SELECT anon_1.targetid AS anon_1_targetid 
FROM (SELECT DISTINCT fuji.target.targetid AS targetid, fuji.target.sv1_desi_target AS sv1_desi_target 
FROM fuji.target 
WHERE fuji.target.targetid IN (SELECT fuji.target.targetid 
FROM fuji.target JOIN fuji.fiberassign ON fuji.target.targetid = fuji.fiberassign.targetid AND fuji.target.tileid = fuji.fiberassign.tileid 
WHERE fuji.target.survey = %(survey_1)s AND fuji.target.program = %(program_1)s GROUP BY fuji.target.targetid 
HAVING count(fuji.target.tileid) > %(count_1)s) AND fuji.target.survey = %(survey_2)s AND fuji.target.program = %(program_2)s) AS anon_1 GROUP BY anon_1.targetid 
HAVING count(anon_1.sv1_desi_target) > %(count_2)s


79

We will call the result of this query `multiple_target`. If we only want to know the number of objects, we're actually done at this stage: the answer is the number of rows of `multiple_target`.  But we can easily get more complete information:

```sql
SELECT t.targetid, t.survey, t.tileid, t.program, t.obsconditions, t.numobs_init, t.priority_init, t.subpriority, t.sv1_desi_target, t.sv1_bgs_target, t.sv1_mws_target, t.sv1_scnd_target, p.ra, p.dec
    FROM fuji.target AS t JOIN fuji.photometry AS p ON t.targetid = p.targetid
    WHERE t.survey = 'sv1' AND t.program = 'dark' AND t.targetid IN (multiple_target) ORDER BY t.targetid, t.tileid;
```

In [7]:
q = db.dbSession.query(db.Target.targetid, db.Target.survey, db.Target.tileid, db.Target.program, db.Target.obsconditions, db.Target.numobs_init, db.Target.priority_init, db.Target.subpriority, db.Target.sv1_desi_target, db.Target.sv1_bgs_target, db.Target.sv1_mws_target, db.Target.sv1_scnd_target, db.Photometry.ra, db.Photometry.dec).join(db.Photometry).filter(db.Target.survey == 'sv1').filter(db.Target.program == 'dark').filter(db.Target.targetid.in_(multiple_target)).order_by(db.Target.targetid, db.Target.tileid)
print(q)
q.all()

SELECT fuji.target.targetid AS fuji_target_targetid, fuji.target.survey AS fuji_target_survey, fuji.target.tileid AS fuji_target_tileid, fuji.target.program AS fuji_target_program, fuji.target.obsconditions AS fuji_target_obsconditions, fuji.target.numobs_init AS fuji_target_numobs_init, fuji.target.priority_init AS fuji_target_priority_init, fuji.target.subpriority AS fuji_target_subpriority, fuji.target.sv1_desi_target AS fuji_target_sv1_desi_target, fuji.target.sv1_bgs_target AS fuji_target_sv1_bgs_target, fuji.target.sv1_mws_target AS fuji_target_sv1_mws_target, fuji.target.sv1_scnd_target AS fuji_target_sv1_scnd_target, fuji.photometry.ra AS fuji_photometry_ra, fuji.photometry.dec AS fuji_photometry_dec 
FROM fuji.target JOIN fuji.photometry ON fuji.photometry.targetid = fuji.target.targetid 
WHERE fuji.target.survey = %(survey_1)s AND fuji.target.program = %(program_1)s AND fuji.target.targetid IN (SELECT anon_1.targetid 
FROM (SELECT DISTINCT fuji.target.targetid AS targetid, fu

[(39628500923060195, 'sv1', 80690, 'dark', 3, 8, 3400, 0.983122124704334, 54047318697103110, 0, 0, 0, 134.200248289668, 31.0107996972334),
 (39628500923060195, 'sv1', 80691, 'dark', 3, 8, 3400, 0.983122124704334, 4665733337124491014, 0, 0, 34359869440, 134.200248289668, 31.0107996972334),
 (39628500939834932, 'sv1', 80690, 'dark', 7, 1, 1050, 0.906294111915661, 2305843026393563136, 0, 81920, 0, 135.224452209346, 31.005952222602),
 (39628500939834932, 'sv1', 80691, 'dark', 1, 100, 2998, 0.906294111915661, 6917529044820951040, 0, 81922, 131072, 135.224452209346, 31.005952222602),
 (39628506119798938, 'sv1', 80690, 'dark', 7, 1, 1100, 0.678005736634616, 2305843052163366912, 0, 33, 0, 135.081459477579, 31.2520285431903),
 (39628506119798938, 'sv1', 80691, 'dark', 7, 100, 2995, 0.678005736634616, 6917529070590754816, 0, 33, 206158430208, 135.081459477579, 31.2520285431903),
 (39628506119800881, 'sv1', 80690, 'dark', 7, 1, 1050, 0.378658012604629, 2305843026393563136, 0, 81920, 0, 135.184801

Now let's find corresponding rows in the `zpix` table. We can reuse the `multiple_target` query from above.

```sql
SELECT id, targetid, z, zwarn
    FROM fuji.zpix
    WHERE targetid IN (multiple_targets) AND survey = 'sv1' AND program = 'dark';
```

In [8]:
multiple_zpix = db.dbSession.query(db.Zpix).filter(db.Zpix.targetid.in_(multiple_target)).filter(db.Zpix.survey == 'sv1').filter(db.Zpix.program == 'dark')
print(multiple_zpix)
multiple_zpix.count()

SELECT fuji.zpix.id AS fuji_zpix_id, fuji.zpix.targetid AS fuji_zpix_targetid, fuji.zpix.survey AS fuji_zpix_survey, fuji.zpix.program AS fuji_zpix_program, fuji.zpix.spgrp AS fuji_zpix_spgrp, fuji.zpix.spgrpval AS fuji_zpix_spgrpval, fuji.zpix.healpix AS fuji_zpix_healpix, fuji.zpix.z AS fuji_zpix_z, fuji.zpix.zerr AS fuji_zpix_zerr, fuji.zpix.zwarn AS fuji_zpix_zwarn, fuji.zpix.chi2 AS fuji_zpix_chi2, fuji.zpix.coeff_0 AS fuji_zpix_coeff_0, fuji.zpix.coeff_1 AS fuji_zpix_coeff_1, fuji.zpix.coeff_2 AS fuji_zpix_coeff_2, fuji.zpix.coeff_3 AS fuji_zpix_coeff_3, fuji.zpix.coeff_4 AS fuji_zpix_coeff_4, fuji.zpix.coeff_5 AS fuji_zpix_coeff_5, fuji.zpix.coeff_6 AS fuji_zpix_coeff_6, fuji.zpix.coeff_7 AS fuji_zpix_coeff_7, fuji.zpix.coeff_8 AS fuji_zpix_coeff_8, fuji.zpix.coeff_9 AS fuji_zpix_coeff_9, fuji.zpix.npixels AS fuji_zpix_npixels, fuji.zpix.spectype AS fuji_zpix_spectype, fuji.zpix.subtype AS fuji_zpix_subtype, fuji.zpix.ncoeff AS fuji_zpix_ncoeff, fuji.zpix.deltachi2 AS fuji_zpix_

79

### Correcting Anomalous Targeting

Now that we know exactly which objects are anomalous, we can try to fix their targeting bits. We want to take the bitwise `OR` of the targeting bits for these objects. We can reuse objects returned by the `multiple_target` query above.  There are a lot of targeting bits, so it's easier to generate the full list programmatically. We're doing metaprogramming!

```sql
SELECT t.targetid, BIT_OR(t.cmx_target) AS cmx_target, BIT_OR(desi_target) AS desi_target, ...
    FROM fuji.target AS t WHERE t.targetid IN (multiple_target) AND t.survey = 'sv1' AND t.program = 'dark' GROUP BY t.targetid;
```

In [9]:
import itertools
table = 'zpix'
surveys = ('', 'sv1', 'sv2', 'sv3')
programs = ('desi', 'bgs', 'mws', 'scnd')
masks = ['cmx_target'] + [('_'.join(p) if p[0] else p[1]) + '_target'
                          for p in itertools.product(surveys, programs)]
inner_columns = ['targetid', 'survey', 'program'] + masks
print("SELECT t.targetid, " + 
      ', '.join([f"BIT_OR(t.{m}) AS {m}" for m in masks]) +
      f" FROM {specprod}.target AS t WHERE t.targetid IN (multiple_target) AND t.survey = 'sv1' AND t.program = 'dark' GROUP BY t.targetid;")
print("db.dbSession.query(db.Target.targetid, " +
      ', '.join([f"func.bit_or(db.Target.{m}).label('{m}')" for m in masks]) +
      ").filter(db.Target.targetid.in_(multiple_target)).filter(db.Target.survey == 'sv1').filter(db.Target.program == 'dark').group_by(db.Target.targetid)")

SELECT t.targetid, BIT_OR(t.cmx_target) AS cmx_target, BIT_OR(t.desi_target) AS desi_target, BIT_OR(t.bgs_target) AS bgs_target, BIT_OR(t.mws_target) AS mws_target, BIT_OR(t.scnd_target) AS scnd_target, BIT_OR(t.sv1_desi_target) AS sv1_desi_target, BIT_OR(t.sv1_bgs_target) AS sv1_bgs_target, BIT_OR(t.sv1_mws_target) AS sv1_mws_target, BIT_OR(t.sv1_scnd_target) AS sv1_scnd_target, BIT_OR(t.sv2_desi_target) AS sv2_desi_target, BIT_OR(t.sv2_bgs_target) AS sv2_bgs_target, BIT_OR(t.sv2_mws_target) AS sv2_mws_target, BIT_OR(t.sv2_scnd_target) AS sv2_scnd_target, BIT_OR(t.sv3_desi_target) AS sv3_desi_target, BIT_OR(t.sv3_bgs_target) AS sv3_bgs_target, BIT_OR(t.sv3_mws_target) AS sv3_mws_target, BIT_OR(t.sv3_scnd_target) AS sv3_scnd_target FROM fuji.target AS t WHERE t.targetid IN (multiple_target) AND t.survey = 'sv1' AND t.program = 'dark' GROUP BY t.targetid;
db.dbSession.query(db.Target.targetid, func.bit_or(db.Target.cmx_target).label('cmx_target'), func.bit_or(db.Target.desi_target).labe

In [10]:
multiple_target_or = db.dbSession.query(db.Target.targetid, func.bit_or(db.Target.cmx_target).label('cmx_target'), func.bit_or(db.Target.desi_target).label('desi_target'), func.bit_or(db.Target.bgs_target).label('bgs_target'), func.bit_or(db.Target.mws_target).label('mws_target'), func.bit_or(db.Target.scnd_target).label('scnd_target'), func.bit_or(db.Target.sv1_desi_target).label('sv1_desi_target'), func.bit_or(db.Target.sv1_bgs_target).label('sv1_bgs_target'), func.bit_or(db.Target.sv1_mws_target).label('sv1_mws_target'), func.bit_or(db.Target.sv1_scnd_target).label('sv1_scnd_target'), func.bit_or(db.Target.sv2_desi_target).label('sv2_desi_target'), func.bit_or(db.Target.sv2_bgs_target).label('sv2_bgs_target'), func.bit_or(db.Target.sv2_mws_target).label('sv2_mws_target'), func.bit_or(db.Target.sv2_scnd_target).label('sv2_scnd_target'), func.bit_or(db.Target.sv3_desi_target).label('sv3_desi_target'), func.bit_or(db.Target.sv3_bgs_target).label('sv3_bgs_target'), func.bit_or(db.Target.sv3_mws_target).label('sv3_mws_target'), func.bit_or(db.Target.sv3_scnd_target).label('sv3_scnd_target')).filter(db.Target.targetid.in_(multiple_target)).filter(db.Target.survey == 'sv1').filter(db.Target.program == 'dark').group_by(db.Target.targetid)

In [11]:
print(multiple_target_or)
multiple_target_or.count()

SELECT fuji.target.targetid AS fuji_target_targetid, bit_or(fuji.target.cmx_target) AS cmx_target, bit_or(fuji.target.desi_target) AS desi_target, bit_or(fuji.target.bgs_target) AS bgs_target, bit_or(fuji.target.mws_target) AS mws_target, bit_or(fuji.target.scnd_target) AS scnd_target, bit_or(fuji.target.sv1_desi_target) AS sv1_desi_target, bit_or(fuji.target.sv1_bgs_target) AS sv1_bgs_target, bit_or(fuji.target.sv1_mws_target) AS sv1_mws_target, bit_or(fuji.target.sv1_scnd_target) AS sv1_scnd_target, bit_or(fuji.target.sv2_desi_target) AS sv2_desi_target, bit_or(fuji.target.sv2_bgs_target) AS sv2_bgs_target, bit_or(fuji.target.sv2_mws_target) AS sv2_mws_target, bit_or(fuji.target.sv2_scnd_target) AS sv2_scnd_target, bit_or(fuji.target.sv3_desi_target) AS sv3_desi_target, bit_or(fuji.target.sv3_bgs_target) AS sv3_bgs_target, bit_or(fuji.target.sv3_mws_target) AS sv3_mws_target, bit_or(fuji.target.sv3_scnd_target) AS sv3_scnd_target 
FROM fuji.target 
WHERE fuji.target.targetid IN (SELE

79

There are a small number of these, so we can just loop over each one, ensuring that only one row in the `zpix` table is updated at a time.

In [ ]:
from sqlalchemy.exc import ProgrammingError
for row in multiple_target_or.all():
    zpix_match = db.dbSession.query(db.Zpix).filter(db.Zpix.targetid == row.targetid).filter(db.Zpix.survey == 'sv1').filter(db.Zpix.program == 'dark')
    print(zpix_match)
    try:
        zpix_update = db.dbSession.query(db.Zpix).filter(db.Zpix.targetid == row.targetid).filter(db.Zpix.survey == 'sv1').filter(db.Zpix.program == 'dark').update({db.Zpix.z: row.cmx_target}).compile()
    except ProgrammingError as e:
        print(e)
        db.dbSession.rollback()
    # print(zpix_match.count())

In [ ]:
db.dbSession.rollback()